# Multi-Objective Algorithm

In this notebook, we will use the Reference Vector Guided Evolutionary Algorithm (**RVEA**) to find the optimal solutions of the **DTLZ2** problem.

In [ ]:
# install evox, skip it if you have already installed evox
try:
    import evox
except ImportError:
    !pip install --disable-pip-version-check --upgrade -q evox
    import evox

In [2]:
import torch
from torch.profiler import ProfilerActivity, profile

from evox.algorithms import RVEA
from evox.core import jit, use_state
from evox.metrics import igd
from evox.problems.numerical import DTLZ2
from evox.workflows import StdWorkflow

## (Optional) Use GPU to run the code
Often, we want to run our code on a GPU because it will run faster. The following code sets the default to run on GPU. Using the CPU is also fine if no device is available.

In [3]:
# Use GPU first to run the code.
torch.set_default_device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.get_default_device())

cuda:0


## Running example: RVEA on DTLZ2 problem
The following code is used to set up the DTLZ2 problem and the RVEA algorithm. More information about the problem and algorithm can be found in the corresponding section of the documentation.

In [8]:
# Init the problem, algorithm and workflow.
prob = DTLZ2(m=3)
pf = prob.pf()
algo = RVEA(pop_size=100, n_objs=3, lb=-torch.zeros(12), ub=torch.ones(12))
workflow = StdWorkflow()
workflow.setup(algo, prob)
workflow.init_step()

# Init the state_step and the jit_state_step
state_step = use_state(lambda: workflow.step)
state = state_step.init_state()
jit_state_step = jit(state_step, trace=True, example_inputs=(state,))

With this setup in place, we can now start to optimize. We set to let the multi-objective algorithm optimize for 100 steps on this problem

In [9]:
# Run the workflow for 100 steps
t = time.time()
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True, profile_memory=True) as prof:
    for i in range(100):
        state = jit_state_step(state)
        fit = state["self.algorithm.fit"]
        fit = fit[~torch.isnan(fit).any(dim=1)]
        if i % 10 == 0:
          print(igd(fit, pf))

tensor(0.5292, device='cuda:0')
tensor(0.1605, device='cuda:0')
tensor(0.0774, device='cuda:0')
tensor(0.0614, device='cuda:0')
tensor(0.0581, device='cuda:0')
tensor(0.0569, device='cuda:0')
tensor(0.0556, device='cuda:0')
tensor(0.0550, device='cuda:0')
tensor(0.0553, device='cuda:0')
tensor(0.0545, device='cuda:0')
